In [2]:
# Instalacje
!pip install pandas torch torchaudio lightning kagglehub scikit-learn ipython soundfile wandb gdown torchcodec

import os
import shutil
import random
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import kagglehub
import gdown
from pathlib import Path
import wandb

# Ustawienie ziarna losowości dla powtarzalności
pl.seed_everything(42)
wandb.login()

Seed set to 42


True

## KOD do uruchamiania na MAD + Generic Audio Dataset

przed uruchomieniem tego skryptu należy wykonać GAS_integration_script.ipnyb

Oba zakładają że forma datasetów w folderze roboczym jest:

dataset:

        DATASET --- tu jest GAS
                Animals
                Birds
                ...

        MAD_dataset 

        noises

        processed_augmented



Przed uruchomieniem tego skryptu, podfolder DATASET (z generic audio sounds) powinien być tak: (skrypt integracyjny powinien tak zrobić)
 
DATASET:

        Animals
        Birds
        Environment
        Vehicles
        gas_test.csv
        gas_train.csv
        gas.csv

***Integracja działania g.collab vs lokalne***


In [3]:
IS_COLAB = os.path.exists('/content')

local = not IS_COLAB ## powinno samo wykryć
if local:
    dataset_path = "dataset"
else:
    dataset_path = "/content/dataset"

In [4]:
# 1. Pobieranie Datasetu MAD
target_dir = dataset_path
if os.path.exists(target_dir) and len(os.listdir(target_dir)) > 0:
    print(f"Dataset już istnieje w '{target_dir}'.")
else:
    print("Pobieranie datasetu MAD...")
    path = kagglehub.dataset_download("junewookim/mad-dataset-military-audio-dataset")
    os.makedirs(target_dir, exist_ok=True)
    shutil.copytree(path, target_dir, dirs_exist_ok=True)
    print("Pobrano dataset MAD.")

# 2. Pobieranie Szumów (z Twojego pliku)
noise_folder = dataset_path + "/noises"
os.makedirs(noise_folder, exist_ok=True)
url = "https://drive.google.com/drive/folders/14Q_0KNDXACkFQ2oTF1T-gnjIaNbNuaKL?usp=sharing"

if not list(Path(noise_folder).glob("*.wav")):
    print("Pobieranie szumów z Google Drive...")
    try:
        gdown.download_folder(url, output=noise_folder, quiet=False, use_cookies=False)
        print("Pobrano szumy.")
    except Exception as e:
        print(f"Błąd pobierania szumów: {e}")
else:
    print(f"Szumy już istnieją w '{noise_folder}'.")

noise_files = list(glob.glob(os.path.join(noise_folder, "*.wav")))
print(f"Liczba dostępnych plików szumu: {len(noise_files)}")


Dataset już istnieje w 'dataset'.
Szumy już istnieją w 'dataset/noises'.
Liczba dostępnych plików szumu: 5


In [5]:
csv_path = dataset_path + "/MAD_dataset/training.csv"
df_full = pd.read_csv(csv_path)

# Mapowanie nazw kolumn
rename_map = {
    'filename': 'path',
    'class': 'label',
    'class_name': 'label'
}
df_full = df_full.rename(columns=rename_map)

# Funkcja naprawiająca ścieżki
def fix_path_MAD(path):
    path = str(path)
    if not path.startswith("MAD_dataset/training/"):
        return os.path.join("MAD_dataset", path)
    return path

df_full['path'] = df_full['path'].apply(fix_path_MAD)
print(f"Załadowano DataFrame: {len(df_full)} plików.")

#print head
print(df_full.head())


Załadowano DataFrame: 6429 plików.
   Unnamed: 0                            path  label  \
0           0  MAD_dataset/training/398/0.wav      3   
1           1  MAD_dataset/training/398/1.wav      3   
2           2  MAD_dataset/training/398/2.wav      3   
3           3  MAD_dataset/training/398/3.wav      3   
4           4  MAD_dataset/training/398/4.wav      3   

                                       youtube title  \
0  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
1  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
2  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
3  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
4  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   

                                        youtube url source  
0  https://www.youtube.com/watch?v=Oh4q6ck6ufc&t=1s    MAD  
1  https://www.youtube.com/watch?v=Oh4q6ck6ufc&t=1s    MAD  
2  https://www.youtube.com/watch?v=Oh4q6ck6ufc&t=1s    MAD  
3  https://www.youtube.com/watch?v=Oh4q6ck6ufc&

In [6]:
df_train_gas = pd.read_csv("dataset/DATASET/gas_train.csv")
df_test_gas  = pd.read_csv("dataset/DATASET/gas_test.csv")

#print train head
print(df_train_gas.head())


df_full_train = pd.concat([df_full, df_train_gas], ignore_index=True)
df_test_mad = pd.read_csv("dataset/MAD_dataset/test.csv")

#fix test mad paths

df_test_mad['path'] = df_test_mad['path'].apply(fix_path_MAD)

df_full_test = pd.concat([df_test_mad, df_test_gas], ignore_index=True)
print(f"Łączna liczba próbek treningowych: {len(df_full_train)}")
print(f"Łączna liczba próbek testowych: {len(df_full_test)}")


                                           path  label label_name source
0              DATASET/Vehicles/bus/bus_282.wav     24        bus    GAS
1     DATASET/Animals/horse/horse_2_part_11.wav     10      horse    GAS
2             DATASET/Vehicles/bus/bus_3366.wav     24        bus    GAS
3  DATASET/Environment/wind/wind_1_part_460.wav     20       wind    GAS
4        DATASET/Birds/crow/crow_4_part_280.wav     12       crow    GAS
Łączna liczba próbek treningowych: 24185
Łączna liczba próbek testowych: 5476


In [7]:
#print new head
print(df_full_train.head())

   Unnamed: 0                            path  label  \
0         0.0  MAD_dataset/training/398/0.wav      3   
1         1.0  MAD_dataset/training/398/1.wav      3   
2         2.0  MAD_dataset/training/398/2.wav      3   
3         3.0  MAD_dataset/training/398/3.wav      3   
4         4.0  MAD_dataset/training/398/4.wav      3   

                                       youtube title  \
0  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
1  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
2  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
3  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   
4  US Army, NATO. Powerful M1A2 Abrams and TR-85 ...   

                                        youtube url source label_name  
0  https://www.youtube.com/watch?v=Oh4q6ck6ufc&t=1s    MAD        NaN  
1  https://www.youtube.com/watch?v=Oh4q6ck6ufc&t=1s    MAD        NaN  
2  https://www.youtube.com/watch?v=Oh4q6ck6ufc&t=1s    MAD        NaN  
3  https://www.youtube.com/watch?v=Oh4

In [8]:


class CachedAudioDataset(Dataset):
    def __init__(self, df, root_dir, noise_files=None, training=True, target_len=150000, expansion_factor=1):
        """
        expansion_factor: Ile razy powielić dataset w jednej epoce.
        Np. expansion_factor=5 sprawi, że dataset 6000 plików będzie "widziany" jako 30000.
        Każda kopia dostanie inną, losową augmentację.
        """
        self.df = df.reset_index(drop=True)
        self.root_dir = os.path.abspath(str(root_dir).strip())
        self.noise_files = noise_files
        self.training = training
        self.target_len = target_len
        self.expansion_factor = expansion_factor if training else 1
        self.target_sr = 48000

        self.labels_to_indices = self.df.groupby('label').groups
        self.all_labels = list(self.labels_to_indices.keys())

        self.sources = self.df['source'].values
        self.source_to_indices = self.df.groupby('source').groups

        self.label_counts = {
            label: len(indices)
            for label, indices in self.labels_to_indices.items()
        }

        self.neg_label_weights = {
            label: 1.0 / count
            for label, count in self.label_counts.items()
        }

        self._neg_labels, self._neg_weights = zip(*self.neg_label_weights.items())
        

        # 1. Cache SZUMÓW
        self.cached_noises = []
        if noise_files:
            print(f"Cache'owanie {len(noise_files)} plików szumu...")
            for nf in noise_files:
                try:
                    wav, sr = torchaudio.load(nf)
                    if sr != self.target_sr: wav = T.Resample(sr, self.target_sr)(wav)
                    if wav.shape[0] > 1: wav = wav.mean(dim=0, keepdim=True)
                    self.cached_noises.append(wav)
                except: pass

        # 2. Cache DANYCH TRENINGOWYCH
        print(f"Ładowanie {len(self.df)} plików treningowych z: {self.root_dir}")
        self.audio_cache = []
        errors = 0
        for i, row in enumerate(self.df.itertuples()):
            csv_path = str(row.path).strip()
            #full_path = os.path.join(self.root_dir, csv_path)
            full_path = self.resolve_path(csv_path)



            try:
                if not os.path.exists(full_path):
                    raise FileNotFoundError("Plik nie istnieje")

                wav, sr = torchaudio.load(full_path)
                if sr != self.target_sr: wav = T.Resample(sr, self.target_sr)(wav)
                if wav.shape[0] > 1: wav = wav.mean(dim=0, keepdim=True)

                if wav.shape[1] < self.target_len:
                    wav = F.pad(wav, (0, self.target_len - wav.shape[1]))
                elif wav.shape[1] > self.target_len:
                    start = (wav.shape[1] - self.target_len) // 2
                    wav = wav[:, start:start+self.target_len]

                self.audio_cache.append(wav)

            except Exception as e:
                errors += 1
                self.audio_cache.append(torch.randn(1, self.target_len) * 0.001)
                if i % 500 == 0:
                    print(f"  błąd ładowania pliku {i}/{len(self.df)}: {full_path} ({e})")

        if errors > 0:
            print(f" uwaga: {errors} plików nie załadowano (wstawiono szum).")

        else:
            print("sukces: Wszystkie pliki w pamięci RAM.")

        if self.training and self.expansion_factor > 1:
            print(f"🚀 DATASET ROZSZERZONY: {len(self.df)} plików -> {len(self)} wirtualnych próbek na epokę.")

    def aggressive_augment(self, waveform):
        # Prosta agresywna augmentacja
        gain = random.uniform(0.5, 1.5)
        waveform = waveform * gain


        #extra masking
        #freq_mask = T.FrequencyMasking(freq_mask_param=5)
        #time_mask = T.TimeMasking(time_mask_param=10)
        #waveform = freq_mask(waveform)
        #waveform = time_mask(waveform)

        if self.cached_noises and random.random() > 0.3:
            noise_wav = random.choice(self.cached_noises)
            sig_len = waveform.shape[1]
            if noise_wav.shape[1] < sig_len:
                repeats = int(sig_len / noise_wav.shape[1]) + 1
                curr_noise = noise_wav.repeat(1, repeats)[:, :sig_len]
            else:
                start = random.randint(0, noise_wav.shape[1] - sig_len)
                curr_noise = noise_wav[:, start:start+sig_len]
            snr_db = random.uniform(5.0, 25.0)
            signal_power = waveform.norm(p=2)
            noise_power = curr_noise.norm(p=2)
            if noise_power > 0:
                snr = 10 ** (snr_db / 20)
                scale = signal_power / (noise_power * snr + 1e-9)
                waveform = waveform + (curr_noise * scale)
        return waveform

    def __len__(self):
        # Dataset udaje, że jest większy niż w rzeczywistości
        return len(self.df) * self.expansion_factor
    
    def resolve_path(self, csv_path):
        return os.path.join(self.root_dir, csv_path)

    def __getitem__(self, idx):
        # Mapujemy wirtualny indeks na prawdziwy
        real_idx = idx % len(self.df)

        wav_a = self.audio_cache[real_idx].clone()
        label_a = self.df.iloc[real_idx]['label']

        if self.training:
            wav_a = self.aggressive_augment(wav_a)

        # Positive
        # idxs_p = self.labels_to_indices[label_a]
        # possible_p = idxs_p.drop(real_idx, errors='ignore')
        # idx_p = random.choice(possible_p) if len(possible_p) > 0 else real_idx

        idxs_p = self.labels_to_indices[label_a]

        if label_a == 5:  # helicopter
            # allow cross-dataset positives
            possible_p = idxs_p.drop(real_idx, errors='ignore')
        else:
            # same-dataset positives only
            source_a = self.sources[real_idx]
            possible_p = [
                i for i in idxs_p
                if i != real_idx and self.sources[i] == source_a
            ]

        idx_p = random.choice(possible_p) if len(possible_p) > 0 else real_idx


        wav_p = self.audio_cache[idx_p].clone()
        if self.training: wav_p = self.aggressive_augment(wav_p)

        # Negative
        neg_labels = [l for l in self._neg_labels if l != label_a]
        neg_weights = [
            self.neg_label_weights[l] for l in neg_labels
        ]

        label_n = random.choices(neg_labels, weights=neg_weights, k=1)[0]


        #label_n = random.choice([l for l in self.all_labels if l != label_a])
        idx_n = random.choice(self.labels_to_indices[label_n])
        wav_n = self.audio_cache[idx_n].clone()
        if self.training: wav_n = self.aggressive_augment(wav_n)

        return wav_a, wav_p, wav_n


In [9]:
class ResNetTripletGPU(pl.LightningModule):
    def __init__(self, df, root_dir, noise_files, margin=0.8, learning_rate=1e-5):
        #changed margin from 1.0 to 0.3 (v1 --> v2)
        super().__init__()
        self.save_hyperparameters(ignore=['df', 'root_dir', 'noise_files'])
        self.df = df
        self.root_dir = root_dir
        self.noise_files = noise_files
        self.val_preds = []
        self.val_targets = []
        self.test_preds = []
        self.test_targets = []
        self.val_d_ap = []
        self.val_d_an = []

        #checking early backbone freezing
        self.freeze_epochs = 1     # warm-up
        self.partial_epochs = 1    # unfreeze layer4 after this


        # 1. Transformacja na GPU
        self.spec_layer = T.MelSpectrogram(
            sample_rate=48000, n_fft=1024, hop_length=512, n_mels=64, f_min=20, f_max=24000
        )
        self.db_layer = T.AmplitudeToDB()

        # 2. Backbone
        self.backbone = resnet18(weights='IMAGENET1K_V1')

        # Dostosowanie ResNet do 1 kanału
        original_conv1 = self.backbone.conv1
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            self.backbone.conv1.weight.data = original_conv1.weight.data.sum(dim=1, keepdim=True)

        # V2 version with more complex head
        self.backbone.fc = nn.Sequential(
            nn.Linear(512, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(256, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(256, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, 128)
        )

        # V3 version even bigger head
        # self.backbone.fc = nn.Sequential(
        #     nn.Linear(512, 1024, bias=False),
        #     nn.BatchNorm1d(1024),
        #     nn.ReLU(inplace=True),
        #     nn.Dropout(0.2),
        #     nn.Linear(1024, 512, bias=False),
        #     nn.BatchNorm1d(512),
        #     nn.ReLU(inplace=True),
        #     nn.Dropout(0.2),
        #     nn.Linear(512, 256, bias=False),
        #     nn.BatchNorm1d(256),
        #     nn.LeakyReLU(0.1, inplace=True),
        #     nn.Dropout(0.2),
        #     nn.Linear(256, 256, bias=False),
        #     nn.BatchNorm1d(256),
        #     nn.LeakyReLU(0.1, inplace=True),
        #     nn.Linear(256, 128)
        # )

        # self.backbone.fc = nn.Sequential(
        #     nn.Linear(512, 512, bias=False),
        #     nn.BatchNorm1d(512),
        #     nn.ReLU(inplace=True),
        #     nn.Dropout(0.3),
        #     nn.Linear(512, 256, bias=False),
        #     nn.ReLU(inplace=True),
        #     nn.Dropout(0.2),
        #     nn.Linear(256, 256, bias=False),
        #     nn.ReLU(inplace=True),
        #     nn.Linear(256, 128)
        # )


        self.loss_fn = nn.TripletMarginLoss(margin=margin, p=2)

        #soft margin with cosine similarity
        #self.loss_fn = nn.TripletMarginWithDistanceLoss(distance_function=lambda x, y: 1 - F.cosine_similarity(x, y))

    def compute_features(self, wav):
        # To dzieje się na GPU!
        spec = self.spec_layer(wav)
        spec = self.db_layer(spec)
        spec = (spec - spec.mean()) / (spec.std() + 1e-6)
        return self.backbone(spec)
    
    def on_train_epoch_start(self):
        if self.current_epoch == self.freeze_epochs:
            # unfreeze layer4 only
            for name, param in self.backbone.named_parameters():
                if "layer4" in name or "fc" in name:
                    param.requires_grad = True
            print("🔓 Unfroze layer4")

        if self.current_epoch == self.partial_epochs:
            self.unfreeze_backbone()
            print("🔓 Unfroze full backbone")

    def forward(self, x):
        return F.normalize(self.compute_features(x), p=2, dim=1)
    
    def on_fit_start(self):
        self.freeze_backbone()
        print("❄️ Zamrożono backbone na początku treningu")

    #// old version, fallback
    # def training_step(self, batch, batch_idx):
    #     wav_a, wav_p, wav_n = batch
    #     emb_a = self(wav_a)
    #     emb_p = self(wav_p)
    #     emb_n = self(wav_n)

    #     loss = self.loss_fn(emb_a, emb_p, emb_n)
    #     acc = (F.pairwise_distance(emb_a, emb_p) < F.pairwise_distance(emb_a, emb_n)).float().mean()
    #     self.log("train_loss", loss, prog_bar=True)
    #     self.log("train_acc", acc, prog_bar=True)
    #     return loss
    #//

    def training_step(self, batch, batch_idx):
        wav_a, wav_p, wav_n = batch

        emb_a = self(wav_a)
        emb_p = self(wav_p)
        emb_n = self(wav_n)

        # distances
        d_ap = F.pairwise_distance(emb_a, emb_p)          # (B,)
        d_an = F.pairwise_distance(emb_a, emb_n)          # (B,)

        # SEMI-HARD NEGATIVE MASK
        # d_ap < d_an < d_ap + margin
        semi_hard_mask = (d_an > d_ap) & (d_an < d_ap + self.loss_fn.margin)

        # fallback to hardest negatives if none are semi-hard
        if semi_hard_mask.any():
            emb_n = emb_n[semi_hard_mask]
            emb_a = emb_a[semi_hard_mask]
            emb_p = emb_p[semi_hard_mask]
        else:
            # batch-hard fallback
            hard_idx = d_an.argmax()
            emb_n = emb_n[hard_idx].unsqueeze(0)
            emb_a = emb_a[hard_idx].unsqueeze(0)
            emb_p = emb_p[hard_idx].unsqueeze(0)

        loss = self.loss_fn(emb_a, emb_p, emb_n)

        acc = (F.pairwise_distance(emb_a, emb_p) <
            F.pairwise_distance(emb_a, emb_n)).float().mean()

        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        wav_a, wav_p, wav_n = batch
        emb_a = self(wav_a)
        emb_p = self(wav_p)
        emb_n = self(wav_n)

        d_ap = F.pairwise_distance(emb_a, emb_p)
        d_an = F.pairwise_distance(emb_a, emb_n)

        self.val_d_ap.append(d_ap.detach())
        self.val_d_an.append(d_an.detach())

        loss = self.loss_fn(emb_a, emb_p, emb_n)

        preds = (d_ap < d_an).long()          # 1 = correct
        targets = torch.ones_like(preds)      # always positive

        self.val_preds.append(preds)
        self.val_targets.append(targets)

        acc = preds.float().mean()

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
    
    def on_validation_epoch_end(self):
        preds = torch.cat(self.val_preds)
        targets = torch.cat(self.val_targets)

        tp = ((preds == 1) & (targets == 1)).sum().float()
        fp = ((preds == 1) & (targets == 0)).sum().float()
        fn = ((preds == 0) & (targets == 1)).sum().float()

        precision = tp / (tp + fp + 1e-9)
        recall = tp / (tp + fn + 1e-9)
        f1 = 2 * precision * recall / (precision + recall + 1e-9)

        self.log("val_f1", f1, prog_bar=True)

        self.val_preds.clear()
        self.val_targets.clear()

        optimizer = self.optimizers()
        lr = optimizer.param_groups[0]["lr"]
        self.log("lr", lr, prog_bar=True)


        d_ap = torch.cat(self.val_d_ap)
        d_an = torch.cat(self.val_d_an)

        gap = d_an.mean() - d_ap.mean()
        violation = (d_an < d_ap + self.loss_fn.margin).float().mean()

        self.log("val_d_ap", d_ap.mean(), prog_bar=True)
        self.log("val_d_an", d_an.mean(), prog_bar=True)
        self.log("val_gap", gap, prog_bar=True)
        self.log("val_margin_violation", violation, prog_bar=True)

        self.val_d_ap.clear()
        self.val_d_an.clear()



    # def test_step(self, batch, batch_idx):
    #     wav_a, wav_p, wav_n = batch
    #     emb_a = self(wav_a)
    #     emb_p = self(wav_p)
    #     emb_n = self(wav_n)

    #     d_ap = F.pairwise_distance(emb_a, emb_p)
    #     d_an = F.pairwise_distance(emb_a, emb_n)

    #     loss = self.loss_fn(emb_a, emb_p, emb_n)

    #     preds = (d_ap < d_an).long()
    #     targets = torch.ones_like(preds)

    #     self.test_preds.append(preds)
    #     self.test_targets.append(targets)

    #     acc = preds.float().mean()

    #     self.log("test_loss", loss)
    #     self.log("test_acc", acc)

    # def on_test_epoch_end(self):
    #     preds = torch.cat(self.test_preds)
    #     targets = torch.cat(self.test_targets)

    #     tp = ((preds == 1) & (targets == 1)).sum().float()
    #     fp = ((preds == 1) & (targets == 0)).sum().float()
    #     fn = ((preds == 0) & (targets == 1)).sum().float()

    #     precision = tp / (tp + fp + 1e-9)
    #     recall = tp / (tp + fn + 1e-9)
    #     f1 = 2 * precision * recall / (precision + recall + 1e-9)

    #     self.log("test_f1", f1)

    #     self.test_preds.clear()
    #     self.test_targets.clear()


    def test_step(self, batch, batch_idx):
        wav, label = batch
        emb = self(wav)

        return {"emb": emb.detach(), "label": label}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, self.parameters()),
            lr=self.hparams.learning_rate
        )

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="min",
            factor=0.5,
            patience=2
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss"
            }
        }

    def train_dataloader(self):
        train_df, _ = train_test_split(self.df, test_size=0.2, random_state=42, stratify=self.df['label'])
        ds = CachedAudioDataset(
            train_df,
            self.root_dir,
            self.noise_files,
            training=True,
            expansion_factor=4
        )
        return DataLoader(ds, batch_size=64, shuffle=True, num_workers=2, persistent_workers=True)

    def val_dataloader(self):
        _, val_df = train_test_split(self.df, test_size=0.2, random_state=42, stratify=self.df['label'])
        ds = CachedAudioDataset(val_df, self.root_dir, noise_files=None, training=False)
        return DataLoader(ds, batch_size=64, shuffle=False, num_workers=2, persistent_workers=True)
    
    def freeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False
        for param in self.backbone.fc.parameters():
            param.requires_grad = True

    def unfreeze_backbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = True

    

    def recall_at_k(embeddings, labels, k=1):
        embeddings = F.normalize(embeddings, dim=1)
        dists = torch.cdist(embeddings, embeddings)

        # exclude self-match
        dists.fill_diagonal_(float('inf'))

        nn_idx = dists.topk(k, largest=False).indices
        nn_labels = labels[nn_idx]

        correct = (nn_labels == labels.unsqueeze(1)).any(dim=1)
        return correct.float().mean().item()

    def mean_average_precision(embeddings, labels):
        embeddings = F.normalize(embeddings, dim=1)
        dists = torch.cdist(embeddings, embeddings)

        N = len(labels)
        ap = []

        for i in range(N):
            dist = dists[i]
            order = torch.argsort(dist)
            order = order[order != i]

            correct = (labels[order] == labels[i]).float()
            if correct.sum() == 0:
                continue

            precision = torch.cumsum(correct, 0) / (torch.arange(len(correct)) + 1)
            ap.append((precision * correct).sum() / correct.sum())

        return torch.stack(ap).mean().item()


    
    def on_test_epoch_end(self):
        emb = torch.cat([x["emb"] for x in self.trainer.callback_metrics])
        labels = torch.cat([x["label"] for x in self.trainer.callback_metrics])

        r1 = self.recall_at_k(emb, labels, 1)
        r5 = self.recall_at_k(emb, labels, 5)
        map_score = self.mean_average_precision(emb, labels)

        self.log("test_recall@1", r1)
        self.log("test_recall@5", r5)
        self.log("test_mAP", map_score)





In [10]:
import datetime

# ==========================================
# KONFIGURACJA UNIKALNEGO TRENINGU
# ==========================================

#OPCJALNIE: customowy dodatek nazy runu
model_big_name = "IR_ResNet_Triplet_combined_datasets_v2"


# 1. Tworzymy unikalną nazwę folderu na podstawie daty i godziny
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"{model_big_name}_{timestamp}"
checkpoint_dir = os.path.join("checkpoints", run_name)

# Tworzymy folder fizycznie
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"📂 Checkpointy z tego treningu trafią do: {checkpoint_dir}")

# ==========================================
# CALLBACKI I LOGGER
# ==========================================

# 2. Checkpoint najlepszego modelu wg Accuracy
checkpoint_best = ModelCheckpoint(
    monitor="val_acc",
    mode="max",
    dirpath=checkpoint_dir,
    filename="best-epoch={epoch:02d}-acc={val_acc:.4f}",
    save_top_k=1,
    auto_insert_metric_name=False
)

# 3. Checkpoint okresowy (co 5 epok)
checkpoint_periodic = ModelCheckpoint(
    dirpath=checkpoint_dir,
    filename="periodic-epoch={epoch:02d}",
    every_n_epochs=5,
    save_last=True,
    save_top_k=-1
)

#early stopping opcjonalnie
early_stopping = pl.callbacks.EarlyStopping(
    monitor="val_acc",
    patience=5,
    mode="max"
)

# 4. WandB Logger
wandb_logger = WandbLogger(
    project="siamese-audio-classifier",
    entity="deep-neural-network-course",
    name=run_name,
    log_model=False
)

# ==========================================
# START TRENINGU
# ==========================================

if local:
    ROOT_DIR = "dataset/"  # upewnij się, że to poprawna ścieżka!
else:
    ROOT_DIR = "/content/dataset/" # upewnij się, że to poprawna ścieżka!


model = ResNetTripletGPU(
    df=df_full_train,
    root_dir=ROOT_DIR,
    noise_files=noise_files,
    margin=0.5
)

trainer = pl.Trainer(
    max_epochs=15,
    accelerator="auto",
    devices="auto",
    logger=wandb_logger,
    callbacks=[checkpoint_best, checkpoint_periodic, early_stopping],
    log_every_n_steps=10,
    precision=32,
    gradient_clip_val=1.0
)

print(f"🚀 Rozpoczynam trening: {run_name}")
trainer.fit(model)

print("Trening zakończony.")
wandb.finish()


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


📂 Checkpointy z tego treningu trafią do: checkpoints/IR_ResNet_Triplet_combined_datasets_v2_2026-01-19_20-31-08
🚀 Rozpoczynam trening: IR_ResNet_Triplet_combined_datasets_v2_2026-01-19_20-31-08


You are using a CUDA device ('NVIDIA GeForce RTX 4050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: WARNING The anonymous setting has no effect and will be removed in a future version.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params | Mode  | FLOPs
-----------------------------------------------------------------
0 | spec_layer | MelSpectrogram    | 0      | train | 0    
1 | db_layer   | AmplitudeToDB     | 0      | train | 0    
2 | backbone   | ResNet            | 11.5 M | train | 0    
3 | loss_fn    | TripletMarginLoss | 0      | train | 0    
-----------------------------------------------------------------
11.5 M    Trainable params
0         Non-trainable params
11.5 M    Total params
45.867    Total estimated model params size (MB)
86        Modules in train mode
0         Modules in eval mode
0         Total Flops


❄️ Zamrożono backbone na początku treningu


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Trening zakończony.


epoch,▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
lr,██████▁▁▁
train_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▇▆▅▆▇▅▆▅▅▄▄▅▄▅▃▄▄▄▄▃▂▃▃▂▃▁▂▁▁▆▂▁▁▂▅▁▁█▃▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_acc,▁▇███████
val_d_an,▇▇█▆▄▄▄▁▂
val_d_ap,█▃▂▂▂▁▁▁▁
val_f1,▁▇███████
val_gap,▁▆█▇▅▆▆▄▄
+2,...


In [11]:
import os
import shutil
import glob
import datetime
import pandas as pd
from torch.utils.data import DataLoader
import time

def build_embedding_bank(model, dataloader, device):
    model.eval()
    embeddings = []
    labels = []

    for wav, label in dataloader:  # NOTE: test dataset must return (wav, label)
        wav = wav.to(device)
        emb = model(wav)
        embeddings.append(emb.cpu())
        labels.append(label)

    embeddings = torch.cat(embeddings)
    labels = torch.cat(labels)
    return embeddings, labels


class CachedEvalAudioDataset(Dataset):
    """
    EVALUATION / TESTING ONLY
    Returns (wav, label)
    Compatible with Recall@K, mAP, NN search
    """

    def __init__(self, df, root_dir, target_len=150000):
        self.df = df.reset_index(drop=True)
        self.root_dir = os.path.abspath(str(root_dir).strip())
        self.target_len = target_len
        self.target_sr = 48000

        print(f"📦 Caching {len(self.df)} evaluation files from: {self.root_dir}")

        self.audio_cache = []
        self.labels = []

        errors = 0
        for row in self.df.itertuples():
            csv_path = str(row.path).strip()
            full_path = os.path.join(self.root_dir, csv_path)

            try:
                wav, sr = torchaudio.load(full_path)

                if sr != self.target_sr:
                    wav = T.Resample(sr, self.target_sr)(wav)

                if wav.shape[0] > 1:
                    wav = wav.mean(dim=0, keepdim=True)

                if wav.shape[1] < self.target_len:
                    wav = F.pad(wav, (0, self.target_len - wav.shape[1]))
                elif wav.shape[1] > self.target_len:
                    start = (wav.shape[1] - self.target_len) // 2
                    wav = wav[:, start:start + self.target_len]

            except Exception:
                errors += 1
                wav = torch.randn(1, self.target_len) * 0.001

            self.audio_cache.append(wav)
            self.labels.append(row.label)

        self.labels = torch.tensor(self.labels)

        if errors > 0:
            print(f"⚠️ {errors} files failed to load (replaced with noise)")
        else:
            print("✅ All evaluation files cached in RAM")

    def __len__(self):
        return len(self.audio_cache)

    def __getitem__(self, idx):
        return self.audio_cache[idx], self.labels[idx]


#best_local_path = "checkpoints/IR_ResNet_Triplet_combined_datasets_v2_2026-01-19_19-42-55/best-epoch=02-acc=0.9516.ckpt"
best_local_path = checkpoint_best.best_model_path
print("path to best model:", best_local_path)
#best_local_path =

Models_path = "Models"
subfolder_name = "Recall_at_K_results_two_datasets"
TARGET_RUN_DIR = os.path.join(Models_path, subfolder_name)


model_pet_name = "two_datasets_final_v1" # np. V1, V2 itp.

print("\n📊 ROZPOCZYNAM TESTOWANIE NAJLEPSZEGO MODELU (Recall@K)...")


ROOT_DIR = "dataset/MAD_dataset" 

test_csv_path = os.path.join(ROOT_DIR, "test.csv")
df_test = pd.read_csv(test_csv_path)

def fix_test_path(path):
    path = str(path)
    if not path.startswith("test/") and not path.startswith("training/"):
        return os.path.join("test", path)
    return path

df_test['path'] = df_test['path'].apply(fix_test_path)

# ✅ NEW: evaluation dataset (no triplets!)
test_ds = CachedEvalAudioDataset(df_test, root_dir=ROOT_DIR)
test_loader = DataLoader(
    test_ds,
    batch_size=64,
    shuffle=False,
    num_workers=2
)




best_model = ResNetTripletGPU.load_from_checkpoint(
    best_local_path,
    df=df_full,
    root_dir=ROOT_DIR,
    noise_files=[]
)
best_model.eval()
best_model.to("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# Embedding extraction
# ==========================================
print("🧠 Ekstrakcja embeddingów...")

device = best_model.device
embeddings = []
labels = []

start_time = datetime.datetime.now()

with torch.no_grad():
    for wav, lbl in test_loader:
        wav = wav.to(device)
        emb = best_model(wav)
        embeddings.append(emb.cpu())
        labels.append(lbl)

embeddings = torch.cat(embeddings)   # [N, D]
labels = torch.cat(labels)           # [N]

end_time = datetime.datetime.now()

time_taken = (end_time - start_time).total_seconds() * 1000
avg_inference_time = time_taken / len(test_ds)

print(f"⏱️ Czas testowania: {time_taken:.2f} ms")
print(f"⏱️ Średni czas inferencji na próbkę: {avg_inference_time:.4f} ms")

# ==========================================
# Recall@K computation
# ==========================================
def recall_at_k(embeddings, labels, k=1):
    emb = F.normalize(embeddings, dim=1)
    sim = emb @ emb.T  # cosine similarity
    sim.fill_diagonal_(-1)

    topk = sim.topk(k, dim=1).indices
    retrieved_labels = labels[topk]

    correct = (retrieved_labels == labels.unsqueeze(1)).any(dim=1)
    return correct.float().mean().item()

from sklearn.metrics import f1_score

def knn_f1(embeddings, labels, k=5):
    emb = F.normalize(embeddings, dim=1)
    sim = emb @ emb.T
    sim.fill_diagonal_(-1)

    knn = sim.topk(k, dim=1).indices
    preds = []

    for i in range(len(labels)):
        neighbor_labels = labels[knn[i]]
        pred = torch.mode(neighbor_labels).values
        preds.append(pred.item())

    return f1_score(labels.numpy(), preds, average="macro")

recall_1 = recall_at_k(embeddings, labels, k=1)
recall_2 = recall_at_k(embeddings, labels, k=2)
recall_5 = recall_at_k(embeddings, labels, k=5)
recall_10 = recall_at_k(embeddings, labels, k=10)

knn_f1_score = knn_f1(embeddings, labels, k=5)

print("\n📝 WYNIKI BEST:")
print(f"Recall@1  = {recall_1:.4f}")
print(f"Recall@2  = {recall_2:.4f}")
print(f"Recall@5  = {recall_5:.4f}")
print(f"Recall@10 = {recall_10:.4f}")
print(f"KNN F1 (k=5) = {knn_f1_score:.4f}")


# Upewnij się, że katalog istnieje
os.makedirs(TARGET_RUN_DIR, exist_ok=True)

best_filename = (
    f"{model_pet_name}"
    f"_R1={recall_1:.4f}"
    f"_R5={recall_5:.4f}"
    f"_R10={recall_10:.4f}"
    f"_time={avg_inference_time:.4f}.ckpt"
)

target_best = os.path.join(TARGET_RUN_DIR, best_filename)

try:
    shutil.copy(best_local_path, target_best)
    print(f"✅ Zapisano NAJLEPSZY model: {best_filename}")
except Exception as e:
    print(f"❌ Błąd kopiowania BEST: {e}")


path to best model: /home/iwo/GSN/siamese-audio-classifier/checkpoints/IR_ResNet_Triplet_combined_datasets_v2_2026-01-19_20-31-08/best-epoch=03-acc=0.9450.ckpt

📊 ROZPOCZYNAM TESTOWANIE NAJLEPSZEGO MODELU (Recall@K)...
📦 Caching 1037 evaluation files from: /home/iwo/GSN/siamese-audio-classifier/dataset/MAD_dataset
✅ All evaluation files cached in RAM
🧠 Ekstrakcja embeddingów...
⏱️ Czas testowania: 1640.17 ms
⏱️ Średni czas inferencji na próbkę: 1.5817 ms

📝 WYNIKI BEST:
Recall@1  = 0.5931
Recall@2  = 0.7223
Recall@5  = 0.8505
Recall@10 = 0.9180
KNN F1 (k=5) = 0.5505
✅ Zapisano NAJLEPSZY model: two_datasets_final_v1_R1=0.5931_R5=0.8505_R10=0.9180_time=1.5817.ckpt
